## EZKL Jupyter Notebook Demo 

TODO: we are going to swap ezkl_lib and ezkl. So the ezkl_lib instances here should be renamed to ezkl instead once this happens.

In [1]:
# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import torch
import ezkl_lib
import os
import json


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)
# TODO: ...

# define input_shape
input_shape = [1, 28, 28]

# generate a random x to feed into the network
x = 0.1 * torch.rand(1, *input_shape, requires_grad=True)

# change from training to inference mode
circuit.eval()

# Get the forward pass output from the circuit
torch_out = circuit(x)

# define the various filenames
onnx_filename = os.path.join("network.onnx")
data_filename = os.path.join("data.json")
witness_filename = os.path.join("witness.json")
settings_filename = os.path.join("settings.json")

# Export the model
torch.onnx.export(
    circuit,                    # model being run
    x,                          # model input (or a tuple for multiple inputs)
    onnx_filename,              # where to save the model (can be a file or file-like object)
    export_params=True,         # store the trained parameter weights inside the model file
    opset_version=10,           # the ONNX version to export the model to
    do_constant_folding=True,   # whether to execute constant folding for optimization
    input_names = ['input'],    # the model's input names
    output_names = ['output'],  # the model's output names
    dynamic_axes={
        'input' : {0 : 'batch_size'},    # variable length axes
        'output' : {0 : 'batch_size'}
    }
)

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

# output the data from the network so that you can produce the witness file used in proving
data = dict(input_shapes = [input_shape],
            input_data = [data_array],
            output_data = [((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
with open(data_filename, "w") as f:
    json.dump(data, f)

# initialize a settings file for ezkl
ezkl_lib.gen_settings(onnx_filename, settings_filename)

# obtain the optimal setup for the settings for "resources" or "accuracy"
ezkl_lib.calibrate_settings(
    data_filename, onnx_filename, settings_filename, "resources")

# get optimal settings
with open(settings_filename, "r") as f:
    settings_data = json.load(f)

# generate the witness file to feed into ezkl, 
# note that this witness.json is different from the input.json
# when generating the witness, ezkl quantizes the values provided
ezkl_lib.gen_witness(
    data=data_filename,
    model=onnx_filename,
    output=witness_filename,
    scale=settings_data["run_args"]["scale"],
    batch_size=None,
    settings_path=settings_filename
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



{'input_data': [[0.04157254844903946,
   0.0793125107884407,
   0.0017964184517040849,
   0.07683127373456955,
   0.009646320715546608,
   0.024570215493440628,
   0.03330745920538902,
   0.012565976940095425,
   0.016182661056518555,
   0.07595496624708176,
   0.09422886371612549,
   0.0763050839304924,
   0.014362270012497902,
   0.06291563808917999,
   0.07024477422237396,
   0.07396133989095688,
   0.029776722192764282,
   0.00913097895681858,
   0.0852627232670784,
   0.031963396817445755,
   0.012513828463852406,
   0.04620949551463127,
   0.018115777522325516,
   0.04767211899161339,
   0.029274387285113335,
   0.08592253178358078,
   0.09463157504796982,
   0.07560000568628311,
   0.0670703873038292,
   0.04820739105343819,
   0.007525408174842596,
   0.027518291026353836,
   0.018275368958711624,
   0.08650560677051544,
   0.08852919191122055,
   0.09534335881471634,
   0.091493159532547,
   0.08242488652467728,
   0.047147490084171295,
   0.09972406178712845,
   0.04927687719

In [2]:
params_path = os.path.join('kzg.params')


res = ezkl_lib.gen_srs(params_path, settings_data["run_args"]["logrows"])

In [3]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK

model_path = os.path.join('network.onnx')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
params_path = os.path.join('kzg.params')

res = ezkl_lib.setup(
        model_path,
        vk_path,
        pk_path,
        params_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [4]:
# GENERATE A PROOF

data_path = os.path.join("input.json")
proof_path = os.path.join('test.pf')

res = ezkl_lib.prove(
        data_path,
        model_path,
        pk_path,
        proof_path,
        params_path,
        "poseidon",
        "single",
        settings_path,
        False
    )

assert res == True
assert os.path.isfile(proof_path)

In [5]:
# VERIFY IT

res = ezkl_lib.verify(
        proof_path,
        settings_path,
        vk_path,
        params_path,
    )

assert res == True
print("verified")

verified
